# FcaBmf running over MovieLens 100K Demo

Copyright 2022 Bernardo C. Rodrigues

This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public
License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later
version. This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the
implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more
details. You should have received a copy of the GNU General Public License along with this program. If not, 
see <https://www.gnu.org/licenses/>. 

In [1]:
import numpy as np
import pandas as pd
from surprise import Dataset
from lib.BooleanMatrixBasedRecomenders import FcaBmf
from lib.BinaryDataset import BinaryDataset
from lib.FormalConceptAnalysis import GreConD

from rich.jupyter import print
from lib.utils import display_as_table

In [2]:
dataset = Dataset.load_builtin("ml-100k")
trainset = dataset.build_full_trainset()

results = []

for threshold in range(1,6):
    binary_dataset = BinaryDataset.load_from_trainset(trainset, threshold=threshold)
    concepts, _ = GreConD(binary_dataset, verbose=True)
    
    result = [threshold, binary_dataset.number_of_trues, binary_dataset.sparsity, len(concepts)]
    results.append(result)

pd.DataFrame(results, columns=['Threshold', "# of True's", "Sparcity", "# of Concepts"])

[GreConD] Mining Formal Concepts...
[GreConD] Binary dataset has 100000 True's (sparcity: 0.06)
[GreConD] Mining Formal Concepts OK
[GreConD] Formal Concepts mined: 1469
[GreConD] Final Concepts Coverage 100.0%
[GreConD] Mining Formal Concepts...
[GreConD] Binary dataset has 93890 True's (sparcity: 0.06)
[GreConD] Mining Formal Concepts OK
[GreConD] Formal Concepts mined: 1435
[GreConD] Final Concepts Coverage 100.0%
[GreConD] Mining Formal Concepts...
[GreConD] Binary dataset has 82520 True's (sparcity: 0.05)
[GreConD] Mining Formal Concepts OK
[GreConD] Formal Concepts mined: 1385
[GreConD] Final Concepts Coverage 100.0%
[GreConD] Mining Formal Concepts...
[GreConD] Binary dataset has 55375 True's (sparcity: 0.03)
[GreConD] Mining Formal Concepts OK
[GreConD] Formal Concepts mined: 1280
[GreConD] Final Concepts Coverage 100.0%
[GreConD] Mining Formal Concepts...
[GreConD] Binary dataset has 21201 True's (sparcity: 0.01)
[GreConD] Mining Formal Concepts OK
[GreConD] Formal Concepts mi

,Threshold,# of True's,Sparcity,# of Concepts
0,1,100000,0.063047,1469
1,2,93890,0.059195,1435
2,3,82520,0.052026,1385
3,4,55375,0.034912,1280
4,5,21201,0.013367,956


In [3]:
# Generate some predictions
from surprise.model_selection import KFold

# Lets create a 80% train / 20% test ratio
kf = KFold(n_splits=5)
fold_generator = kf.split(dataset)
trainset, testset = next(fold_generator)


In [4]:
# thresholds = [1, 2, 3]
# ks = [1]
# coverages = [0.6]
thresholds = [1, 2, 3, 4, 5]
ks = [1, 5, 10, 20, 30, 50, 60]
coverages = [0.6, 0.8, 1.0]


In [5]:
from itertools import product
from surprise.accuracy import mae, rmse


results = []

for threshold, k, coverage in product(thresholds, ks, coverages):
    print(threshold, k, coverage)
    algo = FcaBmf(k, coverage, threshold)
    algo.fit(trainset)

    predictions = algo.test(testset)

    result = [threshold, k, coverage, mae(predictions=predictions, verbose=False)]
    results.append(result)

results = pd.DataFrame(results, columns=['threshold', "k", "coverage", "mae"])
results


1 1 0.6

1 1 0.8

1 1 1.0

1 5 0.6

1 5 0.8

1 5 1.0

1 10 0.6

1 10 0.8

1 10 1.0

KeyboardInterrupt: 

In [ ]:
import plotly.express as px

fig = px.scatter_3d(results, x='threshold', y='k', z='coverage', color='mae')
fig.show()